In [193]:
%matplotlib inline
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import os
from scipy import spatial
from collections import defaultdict
tf.logging.set_verbosity(tf.logging.ERROR)


In [2]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

INFO:tensorflow:Using /var/folders/s_/bvqcbl7954v3yyk7yxn7bhnw0000gn/T/tfhub_modules to cache modules.


In [190]:
#os.chdir('W266/smartThesaurus')
!pwd
sent_1= pd.read_csv('Code/sentence_generator/output.txt', sep = ',', names = ['pair', 'word', 'sentence'])
sent_2= pd.read_csv('Code/sentence_generator/output_2.txt', sep = ',', names = ['pair', 'word', 'sentence'])

/Users/nathanielschub/Desktop/MIDS/W266/smartThesaurus


In [194]:
distances = []       

for i in range(len(np.unique(sent_1.pair))):
    pair_no = i

    sents_1= np.asarray(sent_1.loc[sent_1['pair'] == i].sentence)
    word_1 = np.unique(sent_1.loc()[sent_1['pair'] == i].word)[0]
    word_1_indices = []
    for sent in sents_1:
        x = sent.split()
        word_1_indices.append(x.index(word_1))

    sents_2 = np.asarray(sent_2.loc[sent_2['pair'] == i].sentence)
    word_2 = np.unique(sent_2.loc()[sent_1['pair'] == i].word)[0]
    word_2_indices = []
    for sent in sents_2:
        x = sent.split()
        word_2_indices.append(x.index(word_2))


    embeddings_1 = elmo(sents_1,
    signature="default",
    as_dict=True)["elmo"] 

    embeddings_2 = elmo(sents_2,
    signature="default",
    as_dict=True)["elmo"] 

    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    embed_1 = sess.run(embeddings_1)
    embed_2 = sess.run(embeddings_2)

    target_embeddings_1 = []
    for i in range(len(embed_1)):
        target_embeddings_1.append(embed_1[i][word_1_indices[i]])
        mean_embedding_1 = np.mean(np.asarray(target_embeddings_1), axis = 0)

    target_embeddings_2 = []
    for i in range(len(embed_2)):
        target_embeddings_2.append(embed_2[i][word_2_indices[i]])
        mean_embedding_2 = np.mean(np.asarray(target_embeddings_2), axis = 0)

    distance = spatial.distance.cosine(mean_embedding_1, mean_embedding_2)
    distances.append((pair_no, distance))

    if i%100==0:
        print(i)


In [195]:
print(distances)

[(1, 0.25115305185317993), (2, 0.2510792016983032), (3, 0.508088618516922), (4, 0.342008113861084)]
